# 0. Setup

### Import Required Libaried


In [1]:
import os
import openai

from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient

import dotenv 
#load the environment variables of .env file
%load_ext dotenv
%dotenv

### Setup Azure Credentials


In [2]:
# Setup the required credential for using Azure cognitive search
search_endpoint = f"https://{os.getenv('AZURE_SEARCH_ENDPOINT')}.search.windows.net/"
search_creds = AzureKeyCredential(os.getenv("AZURE_SEARCH_KEY"))
search_client = SearchClient(endpoint=search_endpoint,
                                index_name=os.getenv("AZURE_SEARCH_INDEX"),
                                credential=search_creds)

# Setup the required credential for using Azure OpenAI
openai.api_type = "azure"
openai.api_key = os.getenv("AZURE_OPENAI_KEY")        
openai_Service = os.getenv("AZURE_OPENAI_SERVICE")
openai.api_base = f"https://{openai_Service}.openai.azure.com"
openai.api_version = "2023-05-15"

### Setup Embedding model


In [10]:
def compute_embedding(text):
    return openai.Embedding.create(engine="embedding", input=text)["data"][0]["embedding"]

def nonewlines(s: str) -> str:
    return s.replace('\n', ' ').replace('\r', ' ')

# 1. Using ChatGPT Through calling API

In [6]:
query = "Where is the capital of India?"

messages = [
    {'role' : 'user', 'content' : query }
]

chat_completion = openai.ChatCompletion.create(
    deployment_id="chat",
    model="gpt-35-turbo",
    messages=messages, 
    temperature=0.7, 
    max_tokens=1024, 
    n=1)

chat_content = chat_completion.choices[0].message.content
chat_content

'The capital of India is New Delhi.'

### Modifying the role of ChatGPT 
- Adding system message
- Adding few shot example

In [8]:
query = "Help me write a poem about love."

systemMessage = '''You are a Shakespearean writing assistant who speaks in a Shakespearean style. 
                    You help people come up with creative ideas and content like stories, poems, and songs that use Shakespearean style of writing style, including words like "thou" and "hath”.
                    Here are some example of Shakespeare's style:
                    - Romeo, Romeo! Wherefore art thou Romeo?
                    - Love looks not with the eyes, but with the mind; and therefore is winged Cupid painted blind.
                    - Shall I compare thee to a summer's day? Thou art more lovely and more temperate.'''

messages = [
    {'role' : 'system', 'content' : systemMessage},
    {'role' : 'user', 'content' : 'Please write a short text turning down an invitation to dinner.'},
    {'role' : 'assistant', 'content' : '''Dearest,
                                        Regretfully, I must decline thy invitation.
                                        Prior engagements call me hence. Apologies.'''},
    {'role' : 'user', 'content' : query }
]

chat_completion = openai.ChatCompletion.create(
    deployment_id="chat",
    model="gpt-35-turbo",
    messages=messages, 
    temperature=0.7, 
    max_tokens=1024, 
    n=1)

chat_content = chat_completion.choices[0].message.content
print(chat_content)

In the realm of love, where hearts doth intertwine,
Thine enchanting words, like sweet melodies, combine.
Love, a flame that doth ignite the soul's fire,
A passion that doth lift us higher and higher.

Love looks not with the eyes, but with the mind,
Where Cupid's arrow doth strike, thou shalt find,
A love that doth transcend the bounds of time,
A love that's pure, divine, and so sublime.

Oh, love, thou art a tempest, a gentle breeze,
Thou bringest joy and sorrow with great ease.
Thou art a rose, with petals soft and fair,
But beware, for thorns may lurk with hidden snare.

Shall I compare thee to a summer's day?
Thou art more lovely, more enchanting in every way.
Thy beauty doth outshine the brightest star,
Thine love doth heal and soothe my deepest scar.

Love, like a river, doth flow and meander,
Binding souls together with a love so tender.
Through stormy seas and tranquil shores we tread,
Love's embrace, like a shelter, keeps us fed.

So let us celebrate this love so true,
With e

# 2. Implementing a RAG model

### Obtain related information using vector search

In [13]:
query = "Who is the instructor of the course" #your query
query_vector = compute_embedding(query)

r = search_client.search(query, 
                        top=3, 
                        vector=query_vector, 
                        top_k=50, 
                        vector_fields="embedding")

results = [doc["sourcepage"] + ": " + nonewlines(doc["content"]) for doc in r]

for result in results:
    print(result)

Course_Details-0.pdf:    Calculus • Basic knowledge of partial derivative • Simple vector calculus  • Prior exposed to differential geometry is an advantage  Workload Warning • We have designed this course to have ~150h of study • There is an extensive amount of readings, assignments and coding • If you are not willing or able to spend this amount of time in this semester, drop the course o This is an elective course  Acknowledgements  Most teaching materials in this course are based on work done by others (see reading below). In particular, we would like to thank Stuart Russell, Peter Norvig, Dan Klein, Pieter Abbeel and Christopher Manning for sharing their materials.                COMP3270 Artificial Intelligence Dr. Dirk Schnieders    File generated on 1 September 2023 at 12:06 PM                                                                                                                    Page 2 of 4  Teacher  Instructor Dr. Dirk SCHNIEDERS Senior Lecturer Department of Compu

### Query the GPT model with query + information

In [14]:
systemMessage = """AI Assistant that helps user to answer questions from sources provided. Be brief in your answers.
                    Answer ONLY with the facts listed in the list of sources below. 
                    If there isn't enough information below, say you don't know. Do not generate answers that don't use the sources below. 
                    Each source has a name followed by colon and the actual information, always include the source name for each fact you use in the response. 
                    Use square brackets to reference the source, e.g. [info1.txt]. Don't combine sources, list each source separately, e.g. [info1.txt][info2.pdf].
                """

messages = [
    {'role' : 'system', 'content' : systemMessage},
    {'role' : 'user', 'content' : query + "\n\n Source:" + "\n".join(results)}
]

chat_completion = openai.ChatCompletion.create(
    deployment_id="chat",
    model="gpt-35-turbo",
    messages=messages, 
    temperature=0.7, 
    max_tokens=1024, 
    n=1)

chat_content = chat_completion.choices[0].message.content
print(chat_content)

The instructor of the course is Dr. Dirk Schnieders. [Course_Details-0.pdf]
